In [1]:
import plotly.graph_objects as go
import numpy as np
from dataclasses import dataclass
from typing import List, Dict
from pulp import *

from visualization.event_drawer import draw_daily_events
from visualization.calendar_drawer import draw_calendar
from model.model_creator import create_model
from data.generator import generate_data
np.random.seed(1)


# User input


Users provides input with
* calendar setting
* Fixed events (randomly generated)
* Events to assign for 2 Calendars

MIP maximizes the objective value and produces daily assignment maximizing importance of scheduled events


### Notation

#### Sets
* $ i \in \mathcal{E}$ events
* $ i \in \mathcal{E}^f$ fixed events
* $ c \in \mathcal{C}$ calendars to fill
* $ d \in \mathcal{D}$ days
* $ t \in \mathcal{T}$ hourly time slots

#### Variables
* $z_{i,c,d,t}$ boolean variable, 1 if event $i$ was assigned to calendar $c$, day $d$ and hour $t$


#### Constants
* $K_i$ - duration of event $i$
* $F_{c,d,t} \leq 1$ - number of fixed events in calendar $c$, day $d$ and hour $t$
* $L_{c}$ - daily load (hours) allowed in each calendar

$ \max \sum_{i \in \mathcal{E}} \sum_{c \in \mathcal{C}} \sum_{d \in \mathcal{D}} \sum_{t \in \mathcal{T}} W_i z_{i,c,d,t}$

$s.t. $

$\sum_{i \in \mathcal{E}} z_{i,c,d,t} +\sum_{t' < t, t-t'<K_i -1} \sum_{i \in \mathcal{E}} z_{i,c,d,t'}
+ F_{c,d,t}
\leq 1 \;, \forall c \in \mathcal{C}, d \in \mathcal{D},  t \in \mathcal{T}$ [At maximum one event in each time slot]


$ \sum_{i \in \mathcal{E}} \sum_{t \in \mathcal{T}} z_{i,c,d,t} \leq L_c\;, \forall c \in \mathcal{C}, d \in \mathcal{D}$ [Daily load is respected]

$z_{i,c,d,t} \in \{0,1\}$

In [2]:
events = []
fixed_events = []
calendars,fixed_events,events = generate_data(
    num_calendars=2, fixed_events_num=5, events_num=10
)
days = 7

In [3]:
# fig = draw_calendar(calendars[0],fixed_events[calendars[0]])
# draw_calendar(calendars[1],fixed_events[calendars[1]],fig,5)
# fig.show()

In [4]:
calendar_timeslots = [range(c.day_start_time,c.day_end_time+1) for c in calendars]

In [5]:
fixed_events

{Calendar(name="V's Calendar", day_start_time=9, day_end_time=19, daily_load=6): [Event(name='Coffee Break', min_time=11, max_time=12, possible_days=[4, 3, 1], importance=1, duration=1, repetition=1),
  Event(name='Lunch', min_time=8, max_time=9, possible_days=[0, 2, 1], importance=1, duration=1, repetition=4),
  Event(name='Call with Fred', min_time=14, max_time=16, possible_days=[1, 0, 3], importance=1, duration=2, repetition=4)],
 Calendar(name='Bart', day_start_time=9, day_end_time=20, daily_load=10): [Event(name='Coffee Break', min_time=15, max_time=17, possible_days=[2, 1, 0], importance=1, duration=2, repetition=5),
  Event(name='Call with Fred', min_time=15, max_time=17, possible_days=[3, 0, 4], importance=1, duration=2, repetition=5)]}

In [6]:
from ortools.linear_solver import pywraplp


In [7]:
solver, z =  create_model(
    calendars,
    events,
    fixed_events
)

we have more than 1 event assigned for the same time slot, droping constraint
we have more than 1 event assigned for the same time slot, droping constraint
we have more than 1 event assigned for the same time slot, droping constraint


In [8]:
status = solver.Solve()


In [9]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 10.0


NameError: name 'x' is not defined